In [ ]:
#youbike index.py檔
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()

        
        

def main():
    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False)    
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#更2
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.download_youbike_data()
        except Exception:# as e可取消
            messagebox.showerror("錯誤",f'{e}\n將關閉應用程式\n請稍後再試')
            self.destroy()
            #print(e)取消
        

def main():
    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False) #固定視窗,無法大小   
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#更3
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()

        
        

def main():
    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False)    
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#更4
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()

def on_closing(w:Window):
    print("window關閉")
    t.cancel()
    w.destroy()

t = None
def update_data()->None:
    print("做事")
    global t  #建在文件的t
    t = Timer(20,update_data)
    t.start()        

def main():
    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data()
    window.protocol("WM_DELETE_WINDOW",lambda :on_closing(window))       
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#更5
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()

def on_closing(w:Window):
    print("window關閉")
    t.cancel()
    w.destroy()

t = None
def update_data()->None:
    #datasource.updata_sqlite_data()
    print("做事")
    global t 
    #建在文件的t,global的t為外部t,t寫在def main()外面 
    #t = None,整個文件都可以用
    #t建立一個Timer給他
    t = Timer(20,update_data)
    t.start()        

def main():#window只能用在此function
    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data()
    window.protocol("WM_DELETE_WINDOW",lambda :on_closing(window))       
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#更6
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()

     
t = None
def main():
    def on_closing():
        print("window關閉")
        t.cancel()
        window.destroy()

    
    def update_data()->None:
        datasource.updata_sqlite_data()
        global t
        t = Timer(20,update_data)
        t.start()  

    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data()
    window.protocol("WM_DELETE_WINDOW",on_closing)      
    window.mainloop()

if __name__ == '__main__':
    main()

    


In [ ]:
#youbike datasource.py檔
import requests
import sqlite3

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()
#create sql table
def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT)
        );
        '''
    )
    conn.commit()

def updata_sqlite_data():
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")
    __create_table(conn)
    

In [ ]:
#更2
import requests  # 讓人一進來就看到指令
import sqlite3


def download_youbike_data()->list[dict]:#為list,用dict
    """
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    """
    # import requests 這個function會只用到這裡,所以還是寫在最上面

    youbike_url = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"

    response = requests.get(youbike_url)
    #因為用response.raise__for__status()所以以下指令可取消
    #主要是raise
    # def __加底線為呼叫內部function,例:def __create_table
    '''
    if requests.status_codes == 200:
        # print("下載成功")改寫
        raise Exception("下載失敗")
    else:
        # print("下載失敗")改寫
        raise Exception("下載失敗")
    '''

In [ ]:
#更3
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT)
        );
        '''
    )
    conn.commit()

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    INSERT INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)
    conn.commit()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()

In [ ]:
#更4
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT),
            UNIQUE(站點名稱,更新時間) ON CONFLICT REPLACE 
        );
        '''
    )
    conn.commit()

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    REPLACE INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)
    conn.commit()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()

In [ ]:
#更5
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT),
            UNIQUE(站點名稱,更新時間) ON CONFLICT REPLACE 
        );
        '''
    )
    conn.commit()

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    REPLACE INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)
    conn.commit()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()

In [ ]:
#更7
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT),
            UNIQUE(站點名稱,更新時間) ON CONFLICT REPLACE 
        );
        '''
    )
    conn.commit()

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    REPLACE INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)
    conn.commit()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()
    

In [ ]:
#以下為1101上課檔

In [ ]:
# 1101修改 INDEX.PY
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()

     
#t = None
# def main():
    '''def on_closing():
        print("window關閉")
        t.cancel()
        window.destroy()'''

    
    def update_data()->None:
        #datasource.updata_sqlite_data()
        #global t
        #t = Timer(20,update_data)
        #t.start()  
        window.after(3*60*1000,update_date,w)

    window = Window()
    window.title('台北市youbike2.0')
    window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)
    #window.protocol("WM_DELETE_WINDOW",on_closing)  
    #我註冊這個function來執行    
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#更INDEX.PY 2
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT),
            UNIQUE(站點名稱,更新時間) ON CONFLICT REPLACE 
        );
        '''
    )
    conn.commit()
    cursor.close()
    print("create_table成功")

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    REPLACE INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)    
    conn.commit()
    cursor.close()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")    
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()

def lastest_datetime_data():
    conn = sqlite3.connect("youbike.db")
    cursor = conn.cursor()
    sql = '''
    SELECT 站點名稱,MAX(更新時間) AS 更新時間,行政區,地址,總車輛數,可借,可還
    FROM 台北市youbike
    GROUP BY 站點名稱
    '''
    cursor.execute(sql)
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return rows

In [ ]:
#index更3
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView #4

from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)

        bottomFrame = tk.Frame(self)
        #---------------建立treeView---------------
        self.treeview = ttk.Treeview(bottomFrame,columns=('sna','mday','sarea','ar','tot','sbi','bemp'))
        self.treeview.heading('sna',text='站點名稱')
        self.treeview.heading('mday',text='更新時間')
        self.treeview.heading('sarea',text='行政區')
        self.treeview.heading('ar',text='地址')
        self.treeview.heading('tot',text='總車輛數')
        self.treeview.heading('sbi',text='可借')
        self.treeview.heading('bemp',text='可還')
        self.treeview.pack()
        bottomFrame.pack(pady=30)

 
def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#index更4
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView

from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)

        bottomFrame = tk.Frame(self)
        #---------------建立treeView---------------
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,columns=('sna','mday','sarea','ar','tot','sbi','bemp'))
        self.youbikeTreeView.pack()
        bottomFrame.pack(pady=30)


def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
#index更5
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)

        bottomFrame = tk.Frame(self)
        #---------------建立treeView---------------
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,show="headings",columns=('sna','mday','sarea','ar','tot','sbi','bemp'))
        self.youbikeTreeView.pack()
        bottomFrame.pack(pady=30)

        #-----------更新treeView資料---------------
        lastest_data = datasource.lastest_datetime_data()
        self.youbikeTreeView.update_content(lastest_data)


def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':

In [ ]:
#index 更6
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()           
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)

        bottomFrame = tk.Frame(self)
        #---------------建立treeView---------------
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,show="headings",columns=('sna','mday','sarea','ar','tot','sbi','bemp'))
        self.youbikeTreeView.pack(side='left')
        vsb = ttk.Scrollbar(bottomFrame, orient="vertical", command=self.youbikeTreeView.yview)
        vsb.pack(side='left',fill='y')
        self.youbikeTreeView.configure(yscrollcommand=vsb.set)
        bottomFrame.pack(pady=30)
        print(datasource.search_sitename('三'))
        

        


def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        #-----------更新treeView資料---------------
        lastest_data = datasource.lastest_datetime_data()
        w.youbikeTreeView.update_content(lastest_data)

        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
# 1101修改DATASOURCE.PY 更1
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT),
            UNIQUE(站點名稱,更新時間) ON CONFLICT REPLACE 
        );
        '''
    )
    conn.commit()
    cursor.close()
    print("create_table成功")

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    REPLACE INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)    
    conn.commit()
    cursor.close()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")    
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()

def lastest_datetime_data():
    conn = sqlite3.connect("youbike.db")
    cursor = conn.cursor()
    sql = '''
    SELECT 站點名稱,MAX(更新時間) AS 更新時間,行政區,地址,總車輛數,可借,可還
    FROM 台北市youbike
    GROUP BY 站點名稱
    '''
    cursor.execute(sql)
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return rows

In [ ]:
#data更2
import requests
import sqlite3

__all__ = ['updata_sqlite_data']

def __download_youbike_data()->list[dict]:
    '''
    下載台北市youbike資料2.0
    https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json
    '''
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    response = requests.get(youbike_url)
    response.raise_for_status()
    print("下載成功")
    return response.json()

def __create_table(conn:sqlite3.Connection):    
    cursor = conn.cursor()
    cursor.execute(
        '''
        CREATE TABLE  IF NOT EXISTS 台北市youbike(
            "id"	INTEGER,
            "站點名稱"	TEXT NOT NULL,
            "行政區"	TEXT NOT NULL,
            "更新時間"	TEXT NOT NULL,
            "地址"	TEXT,
            "總車輛數"	INTEGER,
            "可借"	INTEGER,
            "可還"	INTEGER,
            PRIMARY KEY("id" AUTOINCREMENT),
            UNIQUE(站點名稱,更新時間) ON CONFLICT REPLACE 
        );
        '''
    )
    conn.commit()
    cursor.close()
    print("create_table成功")

def __insert_data(conn:sqlite3.Connection,values:list[any])->None:
    cursor = conn.cursor()
    sql = '''
    REPLACE INTO 台北市youbike(站點名稱,行政區,更新時間,地址,總車輛數,可借,可還)
        VALUES(?,?,?,?,?,?,?)
    '''
    cursor.execute(sql,values)    
    conn.commit()
    cursor.close()

def updata_sqlite_data()->None:
    '''
    下載,並更新資料庫
    '''
    data = __download_youbike_data()
    conn = sqlite3.connect("youbike.db")    
    __create_table(conn)
    for item in data:
        __insert_data(conn,[item['sna'],item['sarea'],item['mday'],item['ar'],item['tot'],item['sbi'],item['bemp']])
    conn.close()

def lastest_datetime_data()->list[tuple]:
    conn = sqlite3.connect("youbike.db")
    cursor = conn.cursor()
    sql = '''
    SELECT 站點名稱,MAX(更新時間) AS 更新時間,行政區,地址,總車輛數,可借,可還
    FROM 台北市youbike
    GROUP BY 站點名稱
    '''
    cursor.execute(sql)
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return rows

def search_sitename(word:str) -> list[tuple]:
    conn = sqlite3.connect("youbike.db")
    cursor = conn.cursor()
    sql = '''
        SELECT 站點名稱,MAX(更新時間) AS 更新時間,行政區,地址,總車輛數,可借,可還
        FROM 台北市youbike
        GROUP BY 站點名稱
        HAVING 站點名稱 like ?
        '''
    cursor.execute(sql,[f'%{word}%'])
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    return rows

In [ ]:
#youbikeTreeview.py
from tkinter as tk #2
class YoubikeTreeView(ttk.Treeview): #1
    def __init__(self,parent,**kwargs):#3
        super().__init__(parent,**kwargs)#3
        #跳到index.py做其他設定 
        #-------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')
        #self.pack()

In [ ]:
#youbikeTreeview.py更1
from tkinter as tk #2
class YoubikeTreeView(ttk.Treeview): #1
    def __init__(self,parent,**kwargs):#3
        super().__init__(parent,**kwargs)#3
        #跳到index.py做其他設定 #4
        #-----------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')
        #self.pack()

        #----------設定欄位寬度-----------------
        self.column('sna',width=120)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=250)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)


In [ ]:
#YoubikeTreeView 更2
from tkinter import ttk

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=120)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=250)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)


In [ ]:
#youbikeTreeview更3
from tkinter import ttk

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=120)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=250)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

    def update_content(self,site_datas):
        print(site_datas)

In [ ]:
#youbiketreeview更4
from tkinter import ttk

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=200)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=300)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

    def update_content(self,site_datas):
        #print(site_datas)    
        '''
        更新內容
        '''
        #清除所有內容
        for i in self.get_children():
            self.delete(i)
        
        for site in site_datas:
            self.insert('','end',values=site)



In [ ]:
# 以下為1102修改

In [ ]:
# index 更1

# ttk.middleFrame
# tk.Entry(middleFrame,)
# ttk.labelFrame
# ttk.entry


In [ ]:
# index1102更1
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import datasource
from threading import Timer
from youbikeTreeView import YoubikeTreeView

class Window(tk.Tk):                      
    def __init__(self, **kwargs):          
        super().__init__(**kwargs)         

        #------------------更新資料庫資料--------------------
        try:
            datasource.update_sqlite_data()
        except Exception:                     
            messagebox.showerror('錯誤','將關閉應用程式\n請稍後再試')
            self.destroy()   #關閉視窗
        
        #-------------------建立視窗介面--------------------------

        #-----使用top Frame-----
        topFrame = tk.Frame(self, relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame, text='台北市youbike即時資料', font=('arial', 20), bg='#333333', fg='#ffffff', padx=10, pady=10).pack(padx=20, pady=20)
        topFrame.pack(pady=30)

        #---search Frame -----------1101hw
        searchFrame = tk.Frame(self)
        tk.Label(searchFrame, text='站點名稱:').grid(column=0, row=0)
        self.e = tk.StringVar()
        tk.Entry(searchFrame, width=50, textvariable=self.e).grid(column=1, row=0)
        tk.Button(searchFrame, text="搜尋", state='active',command=self.search_data).grid(column=2, row=0)
        searchFrame.pack()

        #--------bottom Frame-------
        bottomFrame = tk.Frame(self)

        #---------------treeview----------------------------
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,columns=('sna','mday','sarea','ar','tot','sbi','bemp'),show="headings") 
        #加'self.'在整個class中被公開使用
        self.youbikeTreeView.pack(side='left')
        
        #---------------scrollbar---------------------------
        self.scrollbar = ttk.Scrollbar(bottomFrame, orient="vertical", command=self.youbikeTreeView.yview)
        self.scrollbar.pack(side='left',fill='y')
        self.youbikeTreeView.configure(yscrollcommand=self.scrollbar.set)
        #-----bottom frame------
        bottomFrame.pack(pady=30)
        print('下載成功1')

    #----建立搜尋----
    middleFrame = ttk.LableFrame(selftext='站點名稱搜尋')
    tk.Label(middleFrame,text='站點搜尋:').pack(side='left')
    self.search_entry = tk.Entry(middleFrame)
    self.search_entry.pack(sude='left')
    
    '''
    def search_data(self):
        search_term = self.e.get()
        result = datasource.search_sitename(search_term)
        self.youbikeTreeView.search(result = result)
        print('下載成功2')'''

def main():
    def update_data(w:Window)->None:
        datasource.update_sqlite_data()

        #---------------update treeview data--------------------
        lastest_data = datasource.lastest_datetime_data()
        w.youbikeTreeView.update_content(lastest_data)

        window.after(180*1000,update_data,w)
    print('下載成功3')

    window = Window()
    window.title('台北市youbike2.0')
    window.resizable(width=False,height=False)
    update_data(window)
    window.mainloop()
    print('下載成功4')


if __name__ == '__main__':
    main()

In [ ]:
# 1102index 更2
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()           
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)
        #---------------------------------------


        #----------建立搜尋------------------------
        middleFrame = ttk.LabelFrame(self,text='站點')
        tk.Label(middleFrame,text='站點名稱搜尋:').pack(side='left')
        self.search_entry = tk.Entry(middleFrame)
        self.search_entry.pack(side='left')
        middleFrame.pack(expand=True,fill='x')
        #----------------------------------------

        #---------------建立treeView---------------
        bottomFrame = tk.Frame(self)
        
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,show="headings",
                                               columns=('sna','mday','sarea','ar','tot','sbi','bemp'),
                                               height=20)#heig
        self.youbikeTreeView.pack(side='left')
        vsb = ttk.Scrollbar(bottomFrame, orient="vertical", command=self.youbikeTreeView.yview)
        vsb.pack(side='left',fill='y')
        self.youbikeTreeView.configure(yscrollcommand=vsb.set)
        bottomFrame.pack(pady=30,padx=20)
        #-------------------------------------------
        
        

        


def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        #-----------更新treeView資料---------------
        lastest_data = datasource.lastest_datetime_data()
        w.youbikeTreeView.update_content(lastest_data)

        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
# 1102index 更3
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()           
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)
        #---------------------------------------


        #----------建立搜尋------------------------
        middleFrame = ttk.LabelFrame(self,text='')
        tk.Label(middleFrame,text='站點名稱搜尋:').pack(side='left')
        self.search_entry = tk.Entry(middleFrame)
        self.search_entry.pack(side='left')
        middleFrame.pack(fill='x',padx=20)
        #----------------------------------------

        #---------------建立treeView---------------
        bottomFrame = tk.Frame(self)
        
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,show="headings",
                                               columns=('sna','mday','sarea','ar','tot','sbi','bemp'),
                                               height=20)
        self.youbikeTreeView.pack(side='left')
        vsb = ttk.Scrollbar(bottomFrame, orient="vertical", command=self.youbikeTreeView.yview)
        vsb.pack(side='left',fill='y')
        self.youbikeTreeView.configure(yscrollcommand=vsb.set)
        bottomFrame.pack(pady=(0,30),padx=20)
        # bottomFrame 將x=30改為x=0
        #-------------------------------------------
        
        

        


def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        #-----------更新treeView資料---------------
        lastest_data = datasource.lastest_datetime_data()
        w.youbikeTreeView.update_content(lastest_data)

        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
# 1101index 更4
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()           
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)
        #---------------------------------------


        #----------建立搜尋------------------------
        middleFrame = ttk.LabelFrame(self,text='')
        tk.Label(middleFrame,text='站點名稱搜尋:').pack(side='left')
        search_entry = tk.Entry(middleFrame)
        search_entry.bind("<KeyRelease>", self.OnEntryClick)
        search_entry.pack(side='left')        
        middleFrame.pack(fill='x',padx=20)
        #----------------------------------------

        #---------------建立treeView---------------
        bottomFrame = tk.Frame(self)
        
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,show="headings",
                                               columns=('sna','mday','sarea','ar','tot','sbi','bemp'),
                                               height=20)
        self.youbikeTreeView.pack(side='left')
        vsb = ttk.Scrollbar(bottomFrame, orient="vertical", command=self.youbikeTreeView.yview)
        vsb.pack(side='left',fill='y')
        self.youbikeTreeView.configure(yscrollcommand=vsb.set)
        bottomFrame.pack(pady=(0,30),padx=20)
        #-------------------------------------------
    
    def OnEntryClick(self,event):
        searchEntry = event.widget
        #使用者輸入的文字
        input_word = searchEntry.get()
        if input_word == "":
            print("空的")
        else:
            search_data = datasource.search_sitename(word=input_word)
            self.youbikeTreeView.update_content(search_data)
        
  

def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        #-----------更新treeView資料---------------
        lastest_data = datasource.lastest_datetime_data()
        w.youbikeTreeView.update_content(lastest_data)

        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
# 1101index 更5
import tkinter as tk
from tkinter import ttk
from youbikeTreeView import YoubikeTreeView
from tkinter import messagebox
from threading import Timer
import datasource

class Window(tk.Tk):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        #---------更新資料庫資料-----------------#
        try:
            datasource.updata_sqlite_data()
        except Exception:
            messagebox.showerror("錯誤",'網路不正常\n將關閉應用程式\n請稍後再試')
            self.destroy()           
        

        #---------建立介面------------------------
        #print(datasource.lastest_datetime_data())
        topFrame = tk.Frame(self,relief=tk.GROOVE,borderwidth=1)
        tk.Label(topFrame,text="台北市youbike及時資料",font=("arial", 20), bg="#333333", fg='#ffffff',padx=10,pady=10).pack(padx=20,pady=20)
        topFrame.pack(pady=30)
        #---------------------------------------


        #----------建立搜尋------------------------
        middleFrame = ttk.LabelFrame(self,text='')
        tk.Label(middleFrame,text='站點名稱搜尋:').pack(side='left')
        search_entry = tk.Entry(middleFrame)
        search_entry.bind("<KeyRelease>", self.OnEntryClick)
        search_entry.pack(side='left')        
        middleFrame.pack(fill='x',padx=20)
        #----------------------------------------

        #---------------建立treeView---------------
        bottomFrame = tk.Frame(self)
        
        self.youbikeTreeView = YoubikeTreeView(bottomFrame,show="headings",
                                               columns=('sna','mday','sarea','ar','tot','sbi','bemp'),
                                               height=20)
        self.youbikeTreeView.pack(side='left')
        vsb = ttk.Scrollbar(bottomFrame, orient="vertical", command=self.youbikeTreeView.yview)
        vsb.pack(side='left',fill='y')
        self.youbikeTreeView.configure(yscrollcommand=vsb.set)
        bottomFrame.pack(pady=(0,30),padx=20)
        #-------------------------------------------
    
    def OnEntryClick(self,event):
        searchEntry = event.widget
        #使用者輸入的文字
        input_word = searchEntry.get()
        if input_word == "":
            lastest_data = datasource.lastest_datetime_data()
            self.youbikeTreeView.update_content(lastest_data)
        else:
            search_data = datasource.search_sitename(word=input_word)
            self.youbikeTreeView.update_content(search_data)
        
        
        

        


def main():    
    def update_data(w:Window)->None:
        datasource.updata_sqlite_data()
        #-----------更新treeView資料---------------
        lastest_data = datasource.lastest_datetime_data()
        w.youbikeTreeView.update_content(lastest_data)

        window.after(3*60*1000,update_data,w) #每隔3分鐘
          

    window = Window()
    window.title('台北市youbike2.0')
    #window.geometry('600x300')
    window.resizable(width=False,height=False)
    update_data(window)          
    window.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
# 1102youbikeTreeView更1
from tkinter import ttk

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=200)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=300)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

        #----------bind button1-------
        self.bind('<ButtonRelease-1>', self.selectedItem)

    def update_content(self,site_datas):
        '''
        更新內容
        '''
        #清除所有內容
        for i in self.get_children():
            self.delete(i)
        
        for index,site in enumerate(site_datas):
            self.insert('','end',text=f"abc{index}",values=site)


    def selectedItem(self,event):
        selectedItem = self.focus()
        print(selectedItem)
        print(self.item(selectedItem))
        


        

In [ ]:
# 1102下午更2
from tkinter import ttk
from tkinter.simpledialog import Dialog

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        self.parent = parent
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=200)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=300)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

        #----------bind button1-------
        self.bind('<ButtonRelease-1>', self.selectedItem)

    def update_content(self,site_datas):
        '''
        更新內容
        '''
        #清除所有內容
        for i in self.get_children():
            self.delete(i)
        
        for index,site in enumerate(site_datas):
            self.insert('','end',text=f"abc{index}",values=site)


    def selectedItem(self,event): # 1102|
        selectedItem = self.focus()
        print(selectedItem)
        print(self.item(selectedItem))
        get_password = GetPassword(self.parent)# 1102
        


class GetPassword(Dialog): # 1102-1
    pass        

In [ ]:
# 1102下午更3
from tkinter import ttk
from tkinter.simpledialog import Dialog

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        self.parent = parent
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=200)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=300)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

        #----------bind button1-------
        self.bind('<ButtonRelease-1>', self.selectedItem)

    def update_content(self,site_datas):
        '''
        更新內容
        '''
        #清除所有內容
        for i in self.get_children():
            self.delete(i)
        
        for index,site in enumerate(site_datas):
            self.insert('','end',text=f"abc{index}",values=site)


    def selectedItem(self,event):
        selectedItem = self.focus()
        print(selectedItem)
        data_dict = self.item(selectedItem) # 1102
        data_list = data_dict['values'] # 1102
        title = data_list[0] # 1102
        detail = ShowDetail(self.parent,title=title) # 1102
        


class ShowDetail(Dialog): # 1102-2
    pass        

In [ ]:
# 1102下午更3 you
from tkinter import ttk
from tkinter.simpledialog import Dialog

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        self.parent = parent
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=200)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=300)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

        #----------bind button1-------
        self.bind('<ButtonRelease-1>', self.selectedItem)

    def update_content(self,site_datas):
        '''
        更新內容
        '''
        #清除所有內容
        for i in self.get_children():
            self.delete(i)
        
        for index,site in enumerate(site_datas):
            self.insert('','end',text=f"abc{index}",values=site)


    def selectedItem(self,event):
        selectedItem = self.focus()
        print(selectedItem)
        data_dict = self.item(selectedItem)
        data_list = data_dict['values']
        title = data_list[0]
        detail = ShowDetail(self.parent,data=data_list,title=title) # 1102-2
        


class ShowDetail(Dialog):
    def __init__(self,parent,data,**kwargs):# 1102-2
        super().__init__(parent,**kwargs)# 1102-2
        self.data = data    # 1102-2  

In [ ]:
# 1102下午更4
from tkinter import ttk
from tkinter.simpledialog import Dialog

class YoubikeTreeView(ttk.Treeview):
    def __init__(self,parent,**kwargs):
        super().__init__(parent,**kwargs)
        self.parent = parent
        #------設定欄位名稱---------------
        self.heading('sna',text='站點名稱')
        self.heading('mday',text='更新時間')
        self.heading('sarea',text='行政區')
        self.heading('ar',text='地址')
        self.heading('tot',text='總車輛數')
        self.heading('sbi',text='可借')
        self.heading('bemp',text='可還')

        #----------設定欄位寬度------------
        self.column('sna',width=200)
        self.column('mday',width=150)
        self.column('sarea',width=50)
        self.column('ar',width=300)
        self.column('tot',width=50)
        self.column('sbi',width=50)
        self.column('bemp',width=50)

        #----------bind button1-------
        self.bind('<ButtonRelease-1>', self.selectedItem)

    def update_content(self,site_datas):
        '''
        更新內容
        '''
        #清除所有內容
        for i in self.get_children():
            self.delete(i)
        
        for index,site in enumerate(site_datas):
            self.insert('','end',text=f"abc{index}",values=site)


    def selectedItem(self,event):
        selectedItem = self.focus()
        print(selectedItem)
        data_dict = self.item(selectedItem)
        data_list = data_dict['values']
        title = data_list[0]
        detail = ShowDetail(self.parent,data=data_list,title=title)
        #detail = ShowDetail(self.parent,title=title)
        #print(self.item(selectedItem))
        #get_password = GetPassword(self.parent)
        


#class GetPassword(Dialog): 
class ShowDetail(Dialog):
    #pass  
    def __init__(self,parent,data,**kwargs):
        super().__init__(parent,**kwargs)
        #self.data = data             
        self.sna = data[0]
        self.mday = data[1]
        self.sarea = data[2]
        self.ar = data[3]
        self.tot = data[4]
        self.sbi = data[5]
        self.bemp = data[6]

    #master副類別
    def body(self, master):
        #super().body(master)
        mainFrame = tk.Frame(master)
        mainFrame.pack(padx=100,pady=100)

        tk.Label(mainFrame,text="站點名稱").grid(column=0, row=0)
        tk.Label(mainFrame,text="更新時間").grid(column=0, row=1)
        tk.Label(mainFrame,text="行政區").grid(column=0, row=2) 
        tk.Label(mainFrame,text="地址").grid(column=0, row=3)
        tk.Label(mainFrame,text="總量").grid(column=0, row=4)
        tk.Label(mainFrame,text="可借").grid(column=0, row=5)
        tk.Label(mainFrame,text="可還").grid(column=0, row=6)
        
        snaVar = tk.StringVar()
        snaVar.set(self.sna)
        tk.Entry(mainFrame,textvariable=Var,state='disabled').grid(column=1,row=0)



   

In [ ]:
# 1108更改youbikeTreeView
from tkinter import ttk
import tkinter as tk
from tkinter.simpledialog import Dialog


class YoubikeTreeView(ttk.Treeview):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        self.parent = parent
        # ------設定欄位名稱---------------
        self.heading("sna", text="站點名稱")
        self.heading("mday", text="更新時間")
        self.heading("sarea", text="行政區")
        self.heading("ar", text="地址")
        self.heading("tot", text="總車輛數")
        self.heading("sbi", text="可借")
        self.heading("bemp", text="可還")

        # ----------設定欄位寬度------------
        self.column("sna", width=200)
        self.column("mday", width=150)
        self.column("sarea", width=50)
        self.column("ar", width=300)
        self.column("tot", width=50)
        self.column("sbi", width=50)
        self.column("bemp", width=50)

        # ----------bind button1-------
        self.bind("<ButtonRelease-1>", self.selectedItem)

    def update_content(self, site_datas):
        """
        更新內容
        """
        # 清除所有內容
        for i in self.get_children():
            self.delete(i)

        for index, site in enumerate(site_datas):
            self.insert("", "end", text=f"abc{index}", values=site)

    def selectedItem(self, event):
        selectedItem = self.focus()
        print(selectedItem)
        data_dict = self.item(selectedItem)
        data_list = data_dict["values"]
        title = data_list[0]
        detail = ShowDetail(self.parent, data=data_list, title=title)


class ShowDetail(Dialog):
    def __init__(self, parent, data, **kwargs):
        self.sna = data[0]
        self.mday = data[1]
        self.sarea = data[2]
        self.ar = data[3]
        self.tot = data[4]
        self.sbi = data[5]
        self.bemp = data[6]
        super().__init__(parent, **kwargs)

    def body(self, master):
        # 原super().body(master) 取消
        """
        override body,可以自訂body的外觀內容
        """
        mainFrame = tk.Frame(master)
        mainFrame.pack(padx=100, pady=100)
        tk.Label(mainFrame, text="站點名稱").grid(column=0, row=0)
        tk.Label(mainFrame, text="更新時間").grid(column=0, row=1)
        tk.Label(mainFrame, text="行政區").grid(column=0, row=2)
        tk.Label(mainFrame, text="地址").grid(column=0, row=3)
        tk.Label(mainFrame, text="總量").grid(column=0, row=4)
        tk.Label(mainFrame, text="可借").grid(column=0, row=5)
        tk.Label(mainFrame, text="可還").grid(column=0, row=6)
        snaVar = tk.StringVar()
        snaVar.set(self.sna)
        tk.Entry(mainFrame, textvariable=snaVar, state="disabled").grid(column=1, row=0)

        mdayVar = tk.StringVar()
        mdayVar.set(self.mday)
        tk.Entry(mainFrame, textvariable=mdayVar, state="disabled").grid(
            column=1, row=1)

        sareaVar = tk.StringVar()
        sareaVar.set(self.sarea)
        tk.Entry(mainFrame, textvariable=sareaVar, state="disabled").grid(
            column=1, row=2)

        arVar = tk.StringVar()
        arVar.set(self.ar)
        tk.Entry(mainFrame, textvariable=arVar, state="disabled").grid(column=1, row=3)

        totVar = tk.StringVar()
        totVar.set(self.tot)
        tk.Entry(mainFrame, textvariable=totVar, state="disabled").grid(column=1, row=4)

        sbiVar = tk.StringVar()
        sbiVar.set(self.sbi)
        tk.Entry(mainFrame, textvariable=sbiVar, state="disabled").grid(column=1, row=5)

        bempVar = tk.StringVar()
        bempVar.set(self.bemp)
        tk.Entry(mainFrame, textvariable=bempVar, state="disabled").grid(
            column=1, row=6)

    def buttonbox(self):
        """
        override buttonbox,可以自訂body的外觀內容
        """
        box = tk.Frame(self)

        w = tk.Button(box, text="確認", width=10, command=self.ok, default=tk.ACTIVE)
        w.pack(padx=5, pady=(5, 20))

        self.bind("<Return>", self.ok)
        self.bind("<Escape>", self.cancel)

        box.pack()

In [ ]:
# simpledialog.pyi檔  按右鍵看定義及宣告
# 借用到程式中
def buttonbox(self):
    """add standard button box.

    override if you do not want the standard buttons
    """

    box = Frame(self)
    # 加tk自訂  box = tk.Frame(self)
    w = Button(box, text="OK", width=10, command=self.ok, default=ACTIVE)
    # 加tk自訂  w = tk.Button(box, text="確認", width=10, command=self.ok, default=tk.ACTIVE)
    # 將ok改成確認
    w.pack(side=LEFT, padx=5, pady=5)
    # 改寫 w.pack(padx=5, pady=(5, 20))
    # LEFT=常數,不可以改變的值
    w = Button(box, text="Cancel", width=10, command=self.cancel)
    w.pack(side=LEFT, padx=5, pady=5)
    # 上方兩列取消
    self.bind("<Return>", self.ok)
    self.bind("<Escape>", self.cancel)

    box.pack()